In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import monodepthloss
import depthencoder 
DEVICE = torch.device("cuda:0")
# Moving to the gpu: net = Net().to(DEVICE)

In [ ]:
network = depthencoder.depthencoder()
network = network.to(DEVICE)
x = torch.randn(3,1280,720).to(DEVICE)
xout = network(x.view(-1,1,1280,720),x.view(-1,1,1280,720),x.view(-1,1,1280,720))
for tensor in xout:
    print(tensor.size())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
import random
import movieparser
import os
import time
from torchvision.utils import save_image
import torchvision

images = os.listdir('left/skyscraper')

optimizer = optim.Adam(network.parameters(),lr=0.001) 

frames = list(range(len(images)))
frames = frames[1200:-10900] # Remove meaningless title/credit images.
EPOCHS = 1

batch_size = 1
results = []
# network.load_state_dict(torch.load('model2.t'))
# for epoch in range(EPOCHS):
random.shuffle(frames)
frame_subset = []
blockofframes = []
for i, frame in enumerate(frames):
    frame_subset.append(frame)
    if (i+1) % batch_size == 0:
        blockofframes.append(frame_subset)
        frame_subset = []

In [ ]:
testing = False
j = 0
mean = []
f= open(f"results-{int(time.time())}.txt","w+")
for frameset in tqdm(blockofframes):
    LinNm, LinN, LinNp, RoutNm, RoutN, RoutNp = movieparser.getframeset(frameset, images)
    
    network.zero_grad()
    output1 = network(LinNm.type(torch.cuda.FloatTensor), \
                     LinN.type(torch.cuda.FloatTensor), \
                     LinNp.type(torch.cuda.FloatTensor))
    output2 = network(RoutNm.type(torch.cuda.FloatTensor), \
                     RoutN.type(torch.cuda.FloatTensor), \
                     RoutNp.type(torch.cuda.FloatTensor))
    #reconstructed = apply_disparity(LinN.type(torch.cuda.FloatTensor), output)
    if(testing):
        plt.imshow(RoutN[0].cpu().view(720, 1280), 'gray')
        plt.show()
        plt.imshow(output1[-3][0].view(720//4, 1280//4).cpu().detach().numpy(), 'gray')
        plt.show()
        break

    loss_function = monodepthloss.MonodepthLoss(n=4, SSIM_w=0.85,\
            disp_gradient_w=0.1, lr_w=1).to(DEVICE)
    output = []
    for i in range(len(output1)):
        output.append(torch.cat((output1[i], output2[i]), 1))
    loss = loss_function(output,[LinN.type(torch.cuda.FloatTensor),\
                                 RoutN.type(torch.cuda.FloatTensor)])
    loss.backward()
    mean.append(loss.item())
    if j % 100 == 0:
        f.write(f"{round(sum(mean)/len(mean),5)}\n")
        f.flush()
        results.append(sum(mean)/len(mean))
        mean = []
    j += 1
    if j % 5000 == 0:
        thetime = int(time.time())
        torch.save(network.state_dict(), f"model-{thetime}")
        save_image(torch.div(RoutN[0].cpu().view(720, 1280),255), \
                   f'imageout/right-{thetime}.png')
        save_image(output1[-1][0].view(720, 1280).cpu(), \
                   f'imageout/depth-{thetime}.png')
    optimizer.step()
f.close()